## Visualising ocean currents from the IMOS Gridded Sea-level Anomaly dataset

This is a quick example to preview the content of the [Gridded sea level anomaly (GSLA) - Near real time](https://portal.aodn.org.au/search?uuid=0c9eb39c-9cbe-4c6a-8a10-5867087e703a) collection, and plot the surface geostrophic velocity.

#### Requirements

The following Python packages (and their dependencies) are used in this notebook
```
# data access & manipulation
netCDF4
h5netcdf
xarray
s3fs
scipy

# plotting
matplotlib
bokeh
hvplot
holoviews
geoviews
jupyterlab
```

### Files and structure

In [1]:
import s3fs
import xarray as xr

s3 = s3fs.S3FileSystem(anon=True)
ds = xr.open_dataset(s3.open('imos-data/IMOS/OceanCurrent/GSLA/NRT/2023/IMOS_OceanCurrent_HV_20231230T000000Z_GSLA_FV02_NRT.nc'))
ds


<xarray.Dataset>
Dimensions:     (TIME: 1, LONGITUDE: 641, LATITUDE: 351)
Coordinates:
  * TIME        (TIME) datetime64[ns] 2023-12-30
  * LONGITUDE   (LONGITUDE) float64 57.0 57.2 57.4 57.6 ... 184.6 184.8 185.0
  * LATITUDE    (LATITUDE) float64 -60.0 -59.8 -59.6 -59.4 ... 9.4 9.6 9.8 10.0
Data variables:
    start_time  (TIME) datetime64[ns] ...
    end_time    (TIME) datetime64[ns] ...
    GSLA        (TIME, LATITUDE, LONGITUDE) float32 ...
    GSL         (TIME, LATITUDE, LONGITUDE) float32 ...
    UCUR        (TIME, LATITUDE, LONGITUDE) float32 ...
    VCUR        (TIME, LATITUDE, LONGITUDE) float32 ...
    UCUR_MEAN   (TIME, LATITUDE, LONGITUDE) float32 ...
    VCUR_MEAN   (TIME, LATITUDE, LONGITUDE) float32 ...
Attributes: (12/35)
    abstract:                      Gridded (adjusted) sea level anomaly (GSLA...
    acknowledgement:               Any users of IMOS data are required to cle...
    author:                        Pilo, Gabriela S
    author_email:                  gabriela.semolinipilo@csiro.au
    citation:                      The citation in a list of references is: "...
    Conventions:                   CF-1.6,IMOS-1.4
    ...                            ...
    project:                       Integrated Marine Observing System (IMOS)
    references:                    http://imos.aodn.org.au/oceancurrent
    standard_name_vocabulary:      NetCDF Climate and Forecast (CF) Metadata ...
    time_coverage_start:           2023-12-08T07:31:54Z
    time_coverage_end:             2024-01-02T19:30:04Z
    title:                         Gridded Sea Level Anomaly - Australia Region

In [2]:
# The ones we're interested in are UCUR and VCUR
print(ds.UCUR)

<xarray.DataArray 'UCUR' (TIME: 1, LATITUDE: 351, LONGITUDE: 641)>
[224991 values with dtype=float32]
Coordinates:
  * TIME       (TIME) datetime64[ns] 2023-12-30
  * LONGITUDE  (LONGITUDE) float64 57.0 57.2 57.4 57.6 ... 184.6 184.8 185.0
  * LATITUDE   (LATITUDE) float64 -60.0 -59.8 -59.6 -59.4 ... 9.4 9.6 9.8 10.0
Attributes:
    description:    eastward geostrophic velocity derived from GSLA + UCUR_MEAN
    long_name:      total eastward geostrophic velocity
    standard_name:  surface_geostrophic_eastward_sea_water_velocity
    units:          m/s


## Simulate surface drifter

In [3]:
import time
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer
from hvplot import xarray

hv.extension('bokeh')

In [13]:
# add coordinate reference system info so that xarray can interpret it
import cartopy.crs as ccrs
ds.attrs['crs'] = ccrs.PlateCarree() #"WGS84"

# plot a slightly smaller subset to speed up and avoid issues around LON=180...
#ds_sub = ds.sel(TIME="2023-12-30", LATITUDE=slice(-50, 0), LONGITUDE=slice(110, 170))
ds_sub = ds.sel(TIME="2023-12-30", LATITUDE=slice(-44, -34), LONGITUDE=slice(150, 156))
ds_sub

<xarray.Dataset>
Dimensions:     (LONGITUDE: 31, LATITUDE: 51)
Coordinates:
    TIME        datetime64[ns] 2023-12-30
  * LONGITUDE   (LONGITUDE) float64 150.0 150.2 150.4 ... 155.6 155.8 156.0
  * LATITUDE    (LATITUDE) float64 -44.0 -43.8 -43.6 -43.4 ... -34.4 -34.2 -34.0
Data variables:
    start_time  datetime64[ns] ...
    end_time    datetime64[ns] ...
    GSLA        (LATITUDE, LONGITUDE) float32 ...
    GSL         (LATITUDE, LONGITUDE) float32 ...
    UCUR        (LATITUDE, LONGITUDE) float32 ...
    VCUR        (LATITUDE, LONGITUDE) float32 ...
    UCUR_MEAN   (LATITUDE, LONGITUDE) float32 ...
    VCUR_MEAN   (LATITUDE, LONGITUDE) float32 ...
Attributes: (12/36)
    abstract:                      Gridded (adjusted) sea level anomaly (GSLA...
    acknowledgement:               Any users of IMOS data are required to cle...
    author:                        Pilo, Gabriela S
    author_email:                  gabriela.semolinipilo@csiro.au
    citation:                      The citation in a list of references is: "...
    Conventions:                   CF-1.6,IMOS-1.4
    ...                            ...
    references:                    http://imos.aodn.org.au/oceancurrent
    standard_name_vocabulary:      NetCDF Climate and Forecast (CF) Metadata ...
    time_coverage_start:           2023-12-08T07:31:54Z
    time_coverage_end:             2024-01-02T19:30:04Z
    title:                         Gridded Sea Level Anomaly - Australia Region
    crs:                           +proj=eqc +ellps=WGS84 +a=6378137.0 +lon_0...

In [18]:
# hvPlot does provide a .vectorfield() method, but it needs the velocities as magnitude & agle rather than x/y components

U = ds_sub.UCUR  # adding .values would give np.ndarray - here we use a xr.Dataarray, which keeps useful things like dimensions &coordinates
V = ds_sub.VCUR
mag = np.sqrt(U**2 + V**2)
angle = np.arctan2(V, U)
ds_vec = xr.Dataset({'mag': mag, 'angle': angle}, attrs={'crs': ds_sub.crs})

vplt = ds_vec.hvplot.vectorfield(x='LONGITUDE', y='LATITUDE', angle='angle', mag='mag',
                                 geo=True, coastline="50m", projection=ccrs.UTM(zone=56, southern_hemisphere=True),
                                 hover=False, frame_height=500)
vplt.opts(hv.opts.VectorField(magnitude='mag', color='mag'))

:Overlay
   .VectorField.I :VectorField   [LONGITUDE,LATITUDE]   (angle,mag)
   .Coastline.I   :Feature   [Longitude,Latitude]

In [25]:
# For a toy model, let's ignore the actual coordinates and just keep the vectors
ds_sub = ds.sel(TIME="2023-12-30", LATITUDE=slice(-40, -30), LONGITUDE=slice(152, 162))
nx = len(ds_sub.LATITUDE)
ny, nx = ds_sub.UCUR.shape
u = ds_sub.UCUR.values
v = ds_sub.VCUR.values
mag = np.sqrt(u**2 + v**2)
angle = np.arctan2(v, u)

ds_toy = xr.Dataset({'x': ('x', np.linspace(0, nx-1, nx)),
                     'y': ('y', np.linspace(0, ny-1, ny)),
                     'u': (('y', 'x'), u),
                     'v': (('y', 'x'), v),
                     'mag': (('y', 'x'), mag),
                     'angle': (('y', 'x'), angle),
                    }
                   )
ds_toy

<xarray.Dataset>
Dimensions:  (x: 51, y: 51)
Coordinates:
  * x        (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 46.0 47.0 48.0 49.0 50.0
  * y        (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 46.0 47.0 48.0 49.0 50.0
Data variables:
    u        (y, x) float32 0.4898 0.4779 0.506 0.564 ... -0.2448 -0.2763 -0.271
    v        (y, x) float32 -0.2385 -0.03845 0.2104 ... 0.3664 0.2014 0.04685
    mag      (y, x) float32 0.5448 0.4795 0.548 0.7204 ... 0.4406 0.342 0.275
    angle    (y, x) float32 -0.4531 -0.08029 0.3941 0.6715 ... 2.16 2.512 2.97

In [33]:
vec_field = ds_toy.hvplot.vectorfield(x='x', y='y', angle='angle', mag='mag', frame_width=500, aspect=1, hover=False).opts(magnitude='mag', color='mag')

In [11]:
float_state = pd.DataFrame({'x': [], 'y': [], 'u': [], 'v': []}, columns=['x', 'y', 'u', 'v'])
dfstream = Buffer(float_state, length=20, index=False)
curve_dmap = hv.DynamicMap(hv.Curve, streams=[dfstream])
point_dmap = hv.DynamicMap(hv.Points, streams=[dfstream])

In [34]:
(vec_field * curve_dmap * point_dmap).opts(
    opts.Points(color='count', line_color='black', size=5, padding=0.1),
    opts.Curve(line_width=1, color='black'))

:DynamicMap   []
   :Overlay
      .VectorField.I :VectorField   [x,y]   (angle,mag)
      .Curve.I       :Curve   [x]   (y,u,v)
      .Points.I      :Points   [x,y]   (u,v)

In [61]:
def gen_flow(x0, y0, dt=0.1):
    x, y = x0, y0
    while True:
        dp = ds_toy.sel(x=x, y=y, method='nearest')
        u = dp.u.values
        v = dp.v.values
        x += u*dt
        y += v*dt
        yield pd.DataFrame([(x, y, u, v)], columns=['x', 'y', 'u', 'v'])

dfstream.following=False
flow = gen_flow(nx/2, ny/2, dt=0.5)
for i in range(200):
    time.sleep(0.1)
    dfstream.send(next(flow))

In [56]:
next(gf)

,x,y,u,v
0,24.947538,24.947538,-0.2623106,-0.2623106


In [60]:
dfstream.clear()